In [23]:
import random

class JogoDeForca:
     def __init__(self):
         import requests
         url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
         r = requests.get(url, allow_redirects=True)
         if r.status_code==200:
             self.content = str(r.content.decode()).split('\n')
         else:
             print("Erro: ", r.status_code)
     
     def novo_jogo(self, vidas=5):
         self.vidas = vidas
         self.palavra = random.choice(self.content)
         return len(self.palavra)

     def tentar_letra(self, letra):
         if self.vidas > 0:
             if letra in self.palavra:
                 return [idx for idx in range(len(self.palavra)) if self.palavra[idx]==letra]
             else:
                 self.vidas -= 1
                 if self.vidas == 0:
                     print("Fim de jogo!")
                     return False
                 else:
                     return []
         
     def tentar_palavra(self, palavra):
         if self.vidas > 0:
             if self.palavra == palavra:
                 print ("Ganhou!")
                 return True
             else:
                 self.vidas = 0
                 print("Fim de jogo!")
                 return False

jogo = JogoDeForca()
print(jogo.novo_jogo())
print(jogo.palavra)
print(jogo.vidas)
print(jogo.tentar_letra('b'))
print(jogo.tentar_letra('a'))
print(jogo.tentar_letra('w'))
print(jogo.vidas)

8
reeditem
5
[]
[]
[]
2


In [24]:
import requests
import numpy as np
import pandas as pd
import plotly.graph_objects as go

class Jogador:
    def __init__(self, jogo):
        self.jogo = jogo
        self.df = {"vitorias": 0, "derrotas": 0}
        self.vitorias = 0
        self.derrotas = 0

    def palavras(self, tamanho, acertos, erros):
        palavras = []
        for i in self.jogo.content:
            if len(i) == tamanho:
                palavras.append(i)

        filtro = []
        for j in palavras:
            correto = True
            for i in range(len(acertos)):
                if acertos[i] != "-" and j[i] != acertos[i]:
                    correto = False
                    break
            if correto:
                filtro.append(j)
        
        palavras = filtro

        filtro = []
        for l in palavras:
            valida = True
            for letra in erros:
                if letra in l:
                    valida = False
                    break
            if valida:
                filtro.append(l)
        
        return filtro
    
    def freq(self, palavras, tentativas):
        freq = {}
        for j in palavras:
            for l in j:
                if l not in tentativas:
                    freq[l] = freq.get(l, 0) + 1
        
        freq = sorted(freq, key=freq.get, reverse=True)

        return freq
    
    def jogadas(self, jogadas):
        for _ in range(jogadas):
            tam = self.jogo.novo_jogo()
            acertos = ['-' for _ in range(tam)]
            tentadas = set()
            erros = set()
            ganhou = False

            while self.jogo.vidas > 0:
                palavras = self.palavras(tam, acertos, erros)
                
                if not palavras:
                    break  

                letras = self.freq(palavras, tentadas)
                
                if not letras:
                    break  

                letra = letras[0] 
                tentadas.add(letra)

                res = self.jogo.tentar_letra(letra)
                if res:
                    for i in res:
                        acertos[i] = letra
                    if "-" not in acertos:
                        ganhou = True
                        break
                else:
                    erros.add(letra)
            
            if ganhou:
                self.vitorias += 1
                self.df["vitorias"] = self.vitorias
            else:
                self.derrotas += 1
                self.df["derrotas"] = self.derrotas

    def graficos(self):
        data = pd.DataFrame.from_dict(self.df, orient='index', columns=['count']).reset_index()
        fig = go.Figure([go.Bar(x=data['index'], y=data['count'])])
        fig.update_layout(title="vic X derrotas", xaxis_title="res", yaxis_title="quantidade")
        fig.show()


In [25]:
jogo = JogoDeForca()
jogador = Jogador(jogo)
jogador.jogadas(jogadas=100)
jogador.graficos()

Fim de jogo!
Fim de jogo!
Fim de jogo!
Fim de jogo!
